In [ ]:
pip install mltu


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.7 MB/s eta 0:00:00
  Created w

In [ ]:
import zipfile

with zipfile.ZipFile('Models.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
import tensorflow as tf
try: [tf.config.experimental.set_memory_growth(gpu, True) for gpu in tf.config.experimental.list_physical_devices("GPU")]
except: pass

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

from mltu.preprocessors import ImageReader
from mltu.transformers import ImageResizer, LabelIndexer, LabelPadding, ImageShowCV2
from mltu.augmentors import RandomBrightness, RandomRotate, RandomErodeDilate, RandomSharpen
from mltu.annotations.images import CVImage

from mltu.tensorflow.dataProvider import DataProvider
from mltu.tensorflow.losses import CTCloss
from mltu.tensorflow.callbacks import Model2onnx, TrainLogger
from mltu.tensorflow.metrics import CWERMetric

from model import train_model
from configs import ModelConfigs

import os
import tarfile
from tqdm import tqdm
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile


def download_and_unzip(url, extract_to="Datasets", chunk_size=1024*1024):
    http_response = urlopen(url)

    data = b""
    iterations = http_response.length // chunk_size + 1
    for _ in tqdm(range(iterations)):
        data += http_response.read(chunk_size)

    zipfile = ZipFile(BytesIO(data))
    zipfile.extractall(path=extract_to)

dataset_path = os.path.join("Datasets", "IAM_Words")
if not os.path.exists(dataset_path):
    download_and_unzip("https://git.io/J0fjL", extract_to="Datasets")

    file = tarfile.open(os.path.join(dataset_path, "words.tgz"))
    file.extractall(os.path.join(dataset_path, "words"))

dataset, vocab, max_len = [], set(), 0

# Preprocess the dataset by the specific IAM_Words dataset file structure
words = open(os.path.join(dataset_path, "words.txt"), "r").readlines()
for line in tqdm(words):
    if line.startswith("#"):
        continue

    line_split = line.split(" ")
    if line_split[1] == "err":
        continue

    folder1 = line_split[0][:3]
    folder2 = "-".join(line_split[0].split("-")[:2])
    file_name = line_split[0] + ".png"
    label = line_split[-1].rstrip("\n")

    rel_path = os.path.join(dataset_path, "words", folder1, folder2, file_name)
    if not os.path.exists(rel_path):
        print(f"File not found: {rel_path}")
        continue

    dataset.append([rel_path, label])
    vocab.update(list(label))
    max_len = max(max_len, len(label))

# Create a ModelConfigs object to store model configurations
configs = ModelConfigs()

# Save vocab and maximum text length to configs
configs.vocab = "".join(vocab)
configs.max_text_length = max_len
configs.save()

# Create a data provider for the dataset
data_provider = DataProvider(
    dataset=dataset,
    skip_validation=True,
    batch_size=configs.batch_size,
    data_preprocessors=[ImageReader(CVImage)],
    transformers=[
        ImageResizer(configs.width, configs.height, keep_aspect_ratio=False),
        LabelIndexer(configs.vocab),
        LabelPadding(max_word_length=configs.max_text_length, padding_value=len(configs.vocab)),
        ],
)

# Split the dataset into training and validation sets
train_data_provider, val_data_provider = data_provider.split(split = 0.9)

# Augment training data with random brightness, rotation and erode/dilate
train_data_provider.augmentors = [
    RandomBrightness(),
    RandomErodeDilate(),
    RandomSharpen(),
    RandomRotate(angle=10),
    ]

# Creating TensorFlow model architecture
model = train_model(
    input_dim = (configs.height, configs.width, 3),
    output_dim = len(configs.vocab),
)

# Compile the model and print summary
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=configs.learning_rate),
    loss=CTCloss(),
    metrics=[CWERMetric(padding_token=len(configs.vocab))],
)
model.summary(line_length=110)

# Define callbacks
earlystopper = EarlyStopping(monitor="val_CER", patience=20, verbose=1)
checkpoint = ModelCheckpoint(f"{configs.model_path}/model.keras", monitor="val_CER", verbose=1, save_best_only=True, mode="min")
trainLogger = TrainLogger(configs.model_path)
tb_callback = TensorBoard(f"{configs.model_path}/logs", update_freq=1)
reduceLROnPlat = ReduceLROnPlateau(monitor="val_CER", factor=0.9, min_delta=1e-10, patience=10, verbose=1, mode="auto")
model2onnx = Model2onnx(f"{configs.model_path}/model.keras")
earlystopper = EarlyStopping(monitor="val_CER", patience=20, verbose=1, mode='min')

# Train the model
model.fit(
    train_data_provider,
    validation_data=val_data_provider,
    epochs=150,
    callbacks=[earlystopper, checkpoint, trainLogger, reduceLROnPlat, tb_callback, model2onnx],

)

# Save training and validation datasets as csv files
train_data_provider.to_csv(os.path.join(configs.model_path, "train.csv"))
val_data_provider.to_csv(os.path.join(configs.model_path, "val.csv"))
import tensorflow as tf

#import tensorflow as tf

# Configure TensorFlow to use GPU
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            # Set GPU to allocate all memory
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)],
            )
        print(f"Using GPU: {gpus}")
    except RuntimeError as e:
        print(f"Error configuring GPU: {e}")
else:
    print("No GPU detected. Using CPU.")


100%|██████████| 115338/115338 [00:01<00:00, 82016.53it/s]


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                   ┃ Output Shape              ┃          Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)             │ (None, 32, 128, 3)        │                0 │ -                          │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ lambda_2 (Lambda)              │ (None, 32, 128, 3)        │                0 │ input[0][0]                │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ conv2d_46 (Conv2D)             │ (None, 32, 128, 16)       │              448 │ lambda_2[0][0]             │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ batch_normalization_36         │ (None, 32, 128, 16)       │               64 │ conv2d_46[0][0]            │
│ (BatchNormalization)           │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ leaky_re_lu_36 (LeakyReLU)     │ (None, 32, 128, 16)       │                0 │ batch_normalization_36[0]… │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ conv2d_47 (Conv2D)             │ (None, 32, 128, 16)       │            2,320 │ leaky_re_lu_36[0][0]       │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ batch_normalization_37         │ (None, 32, 128, 16)       │               64 │ conv2d_47[0][0]            │
│ (BatchNormalization)           │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ conv2d_48 (Conv2D)             │ (None, 32, 128, 16)       │               64 │ lambda_2[0][0]             │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ add_18 (Add)                   │ (None, 32, 128, 16)       │                0 │ batch_normalization_37[0]… │
│                                │                           │                  │ conv2d_48[0][0]            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ leaky_re_lu_37 (LeakyReLU)     │ (None, 32, 128, 16)       │                0 │ add_18[0][0]               │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ dropout_20 (Dropout)           │ (None, 32, 128, 16)       │                0 │ leaky_re_lu_37[0][0]       │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ conv2d_49 (Conv2D)             │ (None, 16, 64, 16)        │            2,320 │ dropout_20[0][0]           │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ batch_normalization_38         │ (None, 16, 64, 16)        │               64 │ conv2d_49[0][0]            │
│ (BatchNormalization)           │                           │                  │                            │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ leaky_re_lu_38 (LeakyReLU)     │ (None, 16, 64, 16)        │                0 │ batch_normalization_38[0]… │
├────────────────────────────────┼───────────────────────────┼──────────────────┼────────────────────────────┤
│ conv2d_50 (Conv2D)             │ (None, 16, 64, 16)        │            2,320 │ leaky_re_lu_38[0][0]       │
├───

 Total params: 549,455 (2.10 MB)

 Trainable params: 547,983 (2.09 MB)

 Non-trainable params: 1,472 (5.75 KB)

Epoch 1/150
5426/5426 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - CER: 1.0044 - WER: 0.9658 - loss: 15.7461
Epoch 1: val_CER improved from inf to 0.79843, saving model to Models/03_handwriting_recognition/202412101123/model.keras
5426/5426 ━━━━━━━━━━━━━━━━━━━━ 479s 86ms/step - CER: 1.0043 - WER: 0.9658 - loss: 15.7456 - val_CER: 0.7984 - val_WER: 0.9100 - val_loss: 10.0552 - learning_rate: 5.0000e-04
Epoch 2/150
5426/5426 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - CER: 0.7344 - WER: 0.8736 - loss: 9.5509
Epoch 2: val_CER improved from 0.79843 to 0.66557, saving model to Models/03_handwriting_recognition/202412101123/model.keras
5426/5426 ━━━━━━━━━━━━━━━━━━━━ 503s 86ms/step - CER: 0.7343 - WER: 0.8736 - loss: 9.5508 - val_CER: 0.6656 - val_WER: 0.8333 - val_loss: 6.7540 - learning_rate: 5.0000e-04
Epoch 3/150
5426/5426 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - CER: 0.6262 - WER: 0.8102 - loss: 7.4002
Epoch 3: val_CER improved from 0.66557 to 0.58148, saving model to Models/03_handwriting_recognition/2024121

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]


In [ ]:
import cv2
import typing
import numpy as np
from tensorflow.keras.models import load_model
from mltu.utils.text_utils import ctc_decoder, get_cer
from keras.config import enable_unsafe_deserialization

# Enable unsafe deserialization (ensure you trust the source of the model)
enable_unsafe_deserialization()

class ImageToWordModel:
    def __init__(self, model_path: str, char_list: typing.Union[str, list]):
        self.model = load_model(model_path, compile=False)
        self.char_list = char_list
        self.input_shape = self.model.input_shape[1:3]  # Assuming NHWC format

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, self.input_shape[::-1])

        # Normalize image to range [0, 1] if necessary
        image = image.astype(np.float32) / 255.0
        image_pred = np.expand_dims(image, axis=0)

        preds = self.model.predict(image_pred)
        text = ctc_decoder(preds, self.char_list)[0]

        return text


if __name__ == "__main__":
    import pandas as pd
    from tqdm import tqdm
    from mltu.configs import BaseModelConfigs

    configs = BaseModelConfigs.load("Models/03_handwriting_recognition/202412050543/configs.yaml")

    model = ImageToWordModel(model_path=configs.model_path, char_list=configs.vocab)

    df = pd.read_csv("Models/03_handwriting_recognition/202412050543/val.csv").values.tolist()

    accum_cer = []
    for image_path, label in tqdm(df):
        image = cv2.imread(image_path)

        prediction_text = model.predict(image)

        cer = get_cer(prediction_text, label)
        print(f"Image: {image_path}, Label: {label}, Prediction: {prediction_text}, CER: {cer}")

        accum_cer.append(cer)

    print(f"Average CER: {np.average(accum_cer)}")


  0%|          | 0/9646 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step


  0%|          | 1/9646 [00:00<1:54:24,  1.41it/s]

Image: Datasets/IAM_Words/words/d04/d04-111/d04-111-05-07.png, Label: ., Prediction: ., CER: 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Image: Datasets/IAM_Words/words/l07/l07-172/l07-172-10-06.png, Label: me, Prediction: ., CER: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


  0%|          | 3/9646 [00:00<38:02,  4.22it/s]  

...

Image: Datasets/IAM_Words/words/m04/m04-007/m04-007-00-10.png, Label: too, Prediction: ., CER: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Image: Datasets/IAM_Words/words/a05/a05-062/a05-062-08-04.png, Label: avert, Prediction: ., CER: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


 80%|████████  | 7725/9646 [13:56<03:16,  9.76it/s]

Image: Datasets/IAM_Words/words/g06/g06-042a/g06-042a-03-04.png, Label: be, Prediction: ., CER: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


 80%|████████  | 7725/9646 [13:56<03:28,  9.24it/s]


TypeError: 'float' object is not iterable

In [ ]:
from tensorflow.keras.models import load_model

# Define the custom lambda function (use the same function as during training)
def custom_lambda(x):
    return x  # Replace with the actual logic of your Lambda layer

# Load the model with the custom Lambda function
model_path = os.path.join("Models", "03_handwriting_recognition", "202412050543", "model.keras")
model = load_model(model_path, custom_objects={'<lambda>': custom_lambda}, compile=False)


ValueError: The `{arg_name}` of this `Lambda` layer is a Python lambda. Deserializing it is unsafe. If you trust the source of the config artifact, you can override this error by passing `safe_mode=False` to `from_config()`, or calling `keras.config.enable_unsafe_deserialization().

In [ ]:
import cv2

image = cv2.imread(image_path)
if image is None:
    print(f"Failed to load image: {image_path}")
else:
    print("Image loaded successfully.")



Failed to load image: Datasets/IAM_Words/words/d04/d04-111/d04-111-05-07.png


# New Section

In [ ]:
!pip install tensorflow

In [ ]:
pip install tf2onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


# New Section